# Coletando Ofertas do dia com BeautifulSoup

## 1. Pacotes

In [72]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

## 2. Definindo Web Crawler:

In [76]:
#definindo site
url = 'https://www.mercadolivre.com.br'
ofertas_dia = f'{url}/ofertas#c_id=/home/today-promotion-recommendations'

#Trazendo user-agent
head = {'User-Agent': 
        'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Mobile Safari/537.36'}

#Inicializando listas:
title = []
preco_cheio = []
preco_real =[]
desconto_perc = []
df = pd.DataFrame( columns = ['produto', 'preco_cheio','preco_com_desconto', '%desconto'])

#definindo num de paginas para coletar
n = 10
pags_scp = 0
for p in range(1,n+1,1):
    try:
        pagina = f'https://www.mercadolivre.com.br/ofertas?container_id=MLB779362-1&page={p}'
        results = requests.get(pagina, headers = head)
        saida = BeautifulSoup(results.text, 'lxml')
        
    except Exception as e:
        print('Exception: ', e)
        
    else:
        #Conteiners:
        container = saida.find('ol', class_ = 'items_container').find_all('li', class_ = 'promotion-item')
        discount_component = saida.find_all('div', class_ = 'promotion-item__discount-component')
        
        for i in range(0,len(container), 1):
            try:
                title.append(container[i].find('p', class_ ='promotion-item__title').get_text())
            
                preco_cheio.append(
                    discount_component[i].find('s', class_= 'andes-money-amount andes-money-amount-combo__previous-value andes-money-amount--previous andes-money-amount--cents-superscript').find('span', class_= 'andes-money-amount__fraction').get_text()
                )
                
                preco_real.append(
                    discount_component[i].find('div', class_= 'andes-money-amount-combo__main-container').find('span', class_= 'andes-money-amount__fraction').get_text()
                )
                
                desconto_perc.append(discount_component[i].find('span', class_ = 'promotion-item__discount-text').get_text())
            
                #ajustando preços:
                #preco_real = [float(num.replace('.', '')) for num in preco_real]
                #preco_cheio= [float(num.replace('.', '')) for num in preco_cheio]
            except Exception as e2:
                print('erro tipo ', e2)
            finally:
            #Criando df de preços
            df_pag = pd.DataFrame(np.array([title, preco_cheio, preco_real, desconto_perc]).T, columns = ['produto', 'preco_cheio','preco_com_desconto', '%desconto'])
            
            #empilhando informações de pags anteriores
            df = pd.concat([df,df_pag], axis = 0,  ignore_index = True)
        #Atualizando contador
        pags_scp+=1
    finally:
        print('Processo finalizado! Pags Coletadas: ', pags_scp)

Processo finalizado! Pags Coletadas:  1
Processo finalizado! Pags Coletadas:  2
Processo finalizado! Pags Coletadas:  3
Processo finalizado! Pags Coletadas:  4
Processo finalizado! Pags Coletadas:  5
Processo finalizado! Pags Coletadas:  6
Processo finalizado! Pags Coletadas:  7
Processo finalizado! Pags Coletadas:  8
Processo finalizado! Pags Coletadas:  9
Processo finalizado! Pags Coletadas:  9


AttributeError: 'NoneType' object has no attribute 'find'

## 3. Inserindo coletas no banco de dados (Oracle)

In [79]:
import oracledb
from datetime import datetime
# definindo rotina p/ carregar no sql:

# definindo queries:
query = [
    f'''INSERT INTO COMPETITORS_DAILY_OFFERS(produto, preco_cheio, preco_com_desconto, %desconto, dt_oferta) VALUES {', '.join(df.iloc[c,:])},{date.today().strftime(format = '%d-%m-%y')}'''
    for c in df.index
]

try:
    con = oracledb.connect(
        user = input('user')
        password = input('pwd')
        dsn = input('dsn')
    )

    cursor = con.cursor()
    

except Exception as e3:
    print('falha na conexão devido a ', e3)

else:
    for r in df.index:
        cursor.execute(query[r])
    # Commitando inclusões

    cursor.execute('commit')
finally:
    con.close()
    print('conexão fechada')

ModuleNotFoundError: No module named 'oracledb'